In [ ]:
import pandas as pd 
import numpy as np
import difflib as diff 
import ast

In [ ]:
df = pd.read_csv("preprocessed_train.csv",keep_default_na=False)
del df['Unnamed: 0']
df.head()

,textID,text,selected_text,sentiment,text_split,selected_text_split,initial_indice
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"['id', 'have', 'responded', 'if', 'i', 'were',...","['id', 'have', 'responded', 'if', 'i', 'were',...",
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"['soon', 'sad', 'i', 'will', 'miss', 'you', 'h...","['soon', 'sad']",
2,088c60f138,my boss is bullying me...,bullying me,negative,"['my', 'boss', 'is', 'bullying', 'me']","['bullying', 'me']",
3,9642c003ef,what interview! leave me alone,leave me alone,negative,"['what', 'interview', 'leave', 'me', 'alone']","['leave', 'me', 'alone']",
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"['sons', 'of', '<curse>', 'why', 'couldnt', 't...","['sons', 'of', '<curse>']",


The next step in this process is to setup the start and end indices that the model will be trained on. We can set this up using either character indices or word indices,we need to decide which. It doesnt make a ton of sense to predit indices that cut a word in half so doing this based on characters probably wont work

As we moved onto finding the indi``ces we noticed a new error: Our split text columns are not lists, they are strings made to look like a list. When we saved to csv for some reason it converted the lists to strings. We are going to use a library called ast to fix this issue. We plan to literally evaluate each string as a list. The library will read in the string i.e "[1,2,4]" and convert it to [1,2,3] which is usable viable python code. Note: This is usually a terrible terrible idea and we acknowledge this. It is however the simplest solution to the issue we are having and as this is a low stakes evironment primarily for learning we accept the usage of literal_eval. For more information read the following [link](https://stackoverflow.com/questions/1832940/why-is-using-eval-a-bad-practice)

In [ ]:
df["text_split"] = df.text_split.apply(lambda x: ast.literal_eval(x))
df["selected_text_split"] = df.selected_text_split.apply(lambda x: ast.literal_eval(x))
df.head()

,textID,text,selected_text,sentiment,text_split,selected_text_split,initial_indice
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"[id, have, responded, if, i, were, going]","[id, have, responded, if, i, were, going]",
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[soon, sad, i, will, miss, you, here, in, san,...","[soon, sad]",
2,088c60f138,my boss is bullying me...,bullying me,negative,"[my, boss, is, bullying, me]","[bullying, me]",
3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[what, interview, leave, me, alone]","[leave, me, alone]",
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"[sons, of, <curse>, why, couldnt, they, put, t...","[sons, of, <curse>]",


Unsurprisingly this data is pretty dirty, so there are plenty of instances where the selected text is really different from the main text. To deal with this we have told get_close_matches to have no cutoff, just return the closest match regardless of similarity.

In [ ]:
def index_finder(text,selected,index):
    text = list(text)
    selected = list(selected)
    return text.index(diff.get_close_matches(selected[index],text,cutoff=0)[0])
df["initial_indice"] = df.apply(lambda x: index_finder(x.text_split,x.selected_text_split,0),axis=1)
df["final_indice"] = df.apply(lambda x: index_finder(x.text_split,x.selected_text_split,-1),axis=1)
df.head()

,textID,text,selected_text,sentiment,text_split,selected_text_split,initial_indice,final_indice
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"[id, have, responded, if, i, were, going]","[id, have, responded, if, i, were, going]",0,6
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[soon, sad, i, will, miss, you, here, in, san,...","[soon, sad]",0,1
2,088c60f138,my boss is bullying me...,bullying me,negative,"[my, boss, is, bullying, me]","[bullying, me]",3,4
3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[what, interview, leave, me, alone]","[leave, me, alone]",2,4
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"[sons, of, <curse>, why, couldnt, they, put, t...","[sons, of, <curse>]",0,2


In [ ]:
df[df.initial_indice > df.final_indice]

,textID,text,selected_text,sentiment,text_split,selected_text_split,initial_indice,final_indice
134,6903cb08f2,Nice to see you tweeting! It`s Sunday 10th M...,e nice,positive,"[nice, to, see, you, sweeting, its, sunday, <n...","[e, nice]",17,0
166,c78bf59e67,#lichfield #tweetup sounds like fun Hope to...,p sounds like fun,positive,"[lichfield, #tweetup, sounds, like, fun, hope,...","[p, sounds, like, fun]",5,4
321,e5160216a5,ohh my tooth is hurts ohh im sad it very hurts,im sad it very hurts,negative,"[ohh, my, tooth, is, hurts, ohh, im, sad, it, ...","[im, sad, it, very, hurts]",6,4
957,c400a6bf99,"alas, I am moving (like where i`m moving to...",h) wish I could go too,positive,"[alas, i, am, moving, like, where, ibm, moving...","[he, wish, i, could, go, too]",10,8
999,6d2170e60f,I`ve been unlocked for decades now...just not...,".just not lucky, never have been.",negative,"[ive, been, unlocked, for, decades, now, just,...","[just, not, lucky, never, have, been]",6,1
1105,5419aaf31e,Packing up and leaving inlaws house heading ho...,nice we,positive,"[packing, up, and, leaving, in-laws, house, he...","[nice, we]",12,8
1122,902c42e338,Yes...met Jon last yr at an interesting 'Ques...,n interesting,positive,"[yes, met, jon, last, yr, at, an, interesting,...","[n, interesting]",16,7
1180,85769c1ef2,Another HD - my lecturer was gob smacked by ho...,gob smacked by how good our presentation was,positive,"[another, hd, -, my, lecturer, was, gob, smack...","[gob, smacked, by, how, good, our, presentatio...",6,5
1279,fb93940016,haha.. you were actually in violations of som...,"Yeah, I went to bed at 4 and I had to be up at...",negative,"[aha, you, were, actually, in, violations, of,...","[yeah, i, went, to, bed, at, <num>, and, i, ha...",10,6
1374,032d67ca83,ROFLMAO for the funny web portal =D,e funny,positive,"[roflmao, for, the, funny, web, portal, d]","[e, funny]",4,3


The issue here is that in places where the selected text is particularly messy the intial index is actually after the selected index. This is a big issue as we absolutely do not want to be telling our model that it is ok to label and initial index as a final index. Since there were only 293 times where that happens which is relatively few we will choose to drop those samples from the dataset 

In [ ]:
df = df[df.initial_indice < df.final_indice]
df[df.initial_indice > df.final_indice]

,textID,text,selected_text,sentiment,text_split,selected_text_split,initial_indice,final_indice


Now there are none that violate that condition!

We can now move to tokenization and preparing the code for modeling!

In [ ]:
df.to_csv("feature_engineered.csv")